## PrepData:

Takes in a dataframe and an activity and generates a new column for it, as well as splits one dataframe into train, valid, and test contiguously 

In [0]:
#export
class PrepData:
  def __init__(self, dataframe, activity):
    self.dataframe = dataframe
    dataframe['Activity'] = activity
    self.lenTrain = int(len(dataframe)/100*70)
    self.lenValid = self.lenTrain + int(len(dataframe)/100*20)
    self.lenTest = self.lenValid + int(len(dataframe)/100*10)
    self.train = dataframe.iloc[:self.lenTrain]
    self.valid = dataframe.iloc[self.lenTrain:self.lenValid]
    self.test = dataframe.iloc[self.lenValid:]

## Combine Data
Takes in multiple dataframes and appends them to the above train, valid, and test dataframes. Example is for two dataframes, repeat for any amount needed

In [0]:
#export
class CombineData:
  def __init__(self, df1, df2):
    self.train = df1.train.append([df2.train])
    self.valid = df1.valid.append([df2.valid])
    self.test = df1.test.append([df2.test])

## Calculate Hidden Layer Size
![alt text](https://i.imgur.com/OaNtCkK.png)

Pass in this hidden layer array into the layers parameters

In [0]:
#export
def calcHiddenLayer(data, alpha, ip, op, numHiddenLayers):
  return [(len(data.train_ds)//(alpha*(ip+op)))//numHiddenLayers]*numHiddenLayers

## Feature Importance
Pass in a learner, a list of categorical variables, and a list of continuous variables.
Based on https://medium.com/@mp.music93/neural-networks-feature-importance-with-fastai-5c393cf65815

In [0]:
#export
def feature_importance(learner, cat_names, cont_names): 
    loss0=np.array([learner.loss_func(learner.pred_batch(batch=(x,y.to("cpu"))), y.to("cpu")) for x,y in iter(learner.data.valid_dl)]).mean()
    fi=dict()
    types=[cat_names, cont_names]
    for j, t in enumerate(types):
      for i, c in enumerate(t):
        loss=[]
        for x,y in iter(learner.data.valid_dl):
          col=x[j][:,i]    #x[0] da hier cat-vars
          idx = torch.randperm(col.nelement())
          x[j][:,i] = col.view(-1)[idx].view(col.size())
          y=y.to('cpu')
          loss.append(learner.loss_func(learner.pred_batch(batch=(x,y)), y))
        fi[c]=np.array(loss).mean()-loss0
    d = sorted(fi.items(), key=lambda kv: kv[1], reverse=True)
    return pd.DataFrame({'cols': [l for l, v in d], 'imp': np.log1p([v for l, v in d])})

# Random Train Test Split

Use sklearn's train_test_split to return two dataframes, one for my test set one for my train

In [0]:
#export
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1)

# Predicting on test set

I wrote this quick function to help me grade my model on a labeled test set quickly.

In [0]:
#export
def PredictTest(df, learner, column):
  i = 0
  for x in range(len(df)):
    if(str(learner.predict(df.iloc[x])[0]) == df[column].iloc[x]):
      i+=1
  return i/len(df)

In [4]:
!python notebook2script.py Tabular.ipynb

Traceback (most recent call last):
  File "notebook2script.py", line 71, in <module>
    if __name__ == '__main__': fire.Fire(notebook2script)
  File "/usr/local/lib/python3.6/dist-packages/fire/core.py", line 127, in Fire
    component_trace = _Fire(component, args, context, name)
  File "/usr/local/lib/python3.6/dist-packages/fire/core.py", line 366, in _Fire
    component, remaining_args)
  File "/usr/local/lib/python3.6/dist-packages/fire/core.py", line 542, in _CallCallable
    result = fn(*varargs, **kwargs)
  File "notebook2script.py", line 45, in notebook2script
    if not allFiles: notebook2scriptSingle(fname)
  File "notebook2script.py", line 56, in notebook2scriptSingle
    main_dic = json.load(open(fname,'r'))
FileNotFoundError: [Errno 2] No such file or directory: 'Tabular.ipynb'
